# 3. LightGBM1
**Start from the most basic features, and try to improve step by step.**

Kaggle score: 0.9350

## Run name

In [1]:
import time

project_name = 'TalkingdataAFD2018'
step_name = 'LightGBM'
time_str = time.strftime("%Y%m%d_%H%M%S", time.localtime())
run_name = '%s_%s_%s' % (project_name, step_name, time_str)
print('run_name: %s' % run_name)
t0 = time.time()

run_name: TalkingdataAFD2018_LightGBM_20180423_204429


## Important params

In [2]:
date = 6
print('date: ', date)

test_n_rows = None
# test_n_rows = 18790469
# test_n_rows = 10*10000

date:  6


In [3]:
day_rows = {
    0: {
        'n_skiprows': 1,
        'n_rows': 10 * 10000
    },
    6: {
        'n_skiprows': 1,
        'n_rows': 9308568
    },
    7: {
        'n_skiprows': 1 + 9308568,
        'n_rows': 59633310
    },
    8: {
        'n_skiprows': 1 + 9308568 + 59633310,
        'n_rows': 62945075
    },
    9: {
        'n_skiprows': 1 + 9308568 + 59633310 + 62945075,
        'n_rows': 53016937
    }
}
n_skiprows = day_rows[date]['n_skiprows']
n_rows = day_rows[date]['n_rows']

## Import PKGs

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from IPython.display import display

import os
import gc
import time
import random
import zipfile
import h5py
import pickle
import math
from PIL import Image
import shutil

from tqdm import tqdm
import multiprocessing
from multiprocessing import cpu_count

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

random_num = np.random.randint(10000)
print('random_num: %s' % random_num)

random_num: 5537


## Project folders

In [5]:
cwd = os.getcwd()

input_folder = os.path.join(cwd, 'input')
output_folder = os.path.join(cwd, 'output')
model_folder = os.path.join(cwd, 'model')
log_folder = os.path.join(cwd, 'log')
print('input_folder: \t\t\t%s' % input_folder)
print('output_folder: \t\t\t%s' % output_folder)
print('model_folder: \t\t\t%s' % model_folder)
print('log_folder: \t\t\t%s' % log_folder)

train_csv_file = os.path.join(input_folder, 'train.csv')
train_sample_csv_file = os.path.join(input_folder, 'train_sample.csv')
test_csv_file = os.path.join(input_folder, 'test.csv')
sample_submission_csv_file = os.path.join(input_folder, 'sample_submission.csv')

print('\ntrain_csv_file: \t\t%s' % train_csv_file)
print('train_sample_csv_file: \t\t%s' % train_sample_csv_file)
print('test_csv_file: \t\t\t%s' % test_csv_file)
print('sample_submission_csv_file: \t%s' % sample_submission_csv_file)

input_folder: 			D:\bitbucket\kaggle\talkingdata-adtracking-fraud-detection\input
output_folder: 			D:\bitbucket\kaggle\talkingdata-adtracking-fraud-detection\output
model_folder: 			D:\bitbucket\kaggle\talkingdata-adtracking-fraud-detection\model
log_folder: 			D:\bitbucket\kaggle\talkingdata-adtracking-fraud-detection\log

train_csv_file: 		D:\bitbucket\kaggle\talkingdata-adtracking-fraud-detection\input\train.csv
train_sample_csv_file: 		D:\bitbucket\kaggle\talkingdata-adtracking-fraud-detection\input\train_sample.csv
test_csv_file: 			D:\bitbucket\kaggle\talkingdata-adtracking-fraud-detection\input\test.csv
sample_submission_csv_file: 	D:\bitbucket\kaggle\talkingdata-adtracking-fraud-detection\input\sample_submission.csv


## Load data

In [6]:
%%time
train_csv = pd.read_csv(train_csv_file, skiprows=range(1, n_skiprows), nrows=n_rows, parse_dates=['click_time'])
test_csv = pd.read_csv(test_csv_file, nrows=test_n_rows, parse_dates=['click_time'])
sample_submission_csv = pd.read_csv(sample_submission_csv_file)

print('train_csv.shape: \t\t', train_csv.shape)
print('test_csv.shape: \t\t', test_csv.shape)
print('sample_submission_csv.shape: \t', sample_submission_csv.shape)
print('train_csv.dtypes: \n', train_csv.dtypes)

display(train_csv.head(2))
display(test_csv.head(2))
display(sample_submission_csv.head(2))

train_csv.shape: 		 (9308568, 8)
test_csv.shape: 		 (18790469, 7)
sample_submission_csv.shape: 	 (18790469, 2)
train_csv.dtypes: 
 ip                          int64
app                         int64
device                      int64
os                          int64
channel                     int64
click_time         datetime64[ns]
attributed_time            object
is_attributed               int64
dtype: object


,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,83230,3,1,13,379,2017-11-06 14:32:21,NaN,0
1,17357,3,1,19,379,2017-11-06 14:33:34,NaN,0


,click_id,ip,app,device,os,channel,click_time
0,0,5744,9,1,3,107,2017-11-10 04:00:00
1,1,119901,9,1,3,466,2017-11-10 04:00:00


,click_id,is_attributed
0,0,0
1,1,0


Wall time: 37 s


In [7]:
y_data = train_csv['is_attributed']
train_csv.drop(['is_attributed'], axis=1, inplace=True)
display(y_data.head())

0    0
1    0
2    0
3    0
4    0
Name: is_attributed, dtype: int64

## Features

## Prepare data

In [8]:
train_useless_features = ['click_time', 'attributed_time']
train_csv.drop(train_useless_features, axis=1, inplace=True)

test_useless_features = ['click_time', 'click_id']
test_csv.drop(test_useless_features, axis=1, inplace=True)

display(train_csv.head())
display(test_csv.head())

,ip,app,device,os,channel
0,83230,3,1,13,379
1,17357,3,1,19,379
2,35810,3,1,13,379
3,45745,14,1,13,478
4,161007,3,1,13,379


,ip,app,device,os,channel
0,5744,9,1,3,107
1,119901,9,1,3,466
2,72287,21,1,19,128
3,78477,15,1,13,111
4,123080,12,1,13,328


In [9]:
x_train, x_val, y_train, y_val = train_test_split(train_csv, y_data, test_size=0.01, random_state=2017)
x_test = test_csv
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)
print(x_test.shape)

(9215482, 5)
(9215482,)
(93086, 5)
(93086,)
(18790469, 5)


## Train

In [10]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score

lgb_train = lgb.Dataset(x_train, label=y_train)
lgb_val = lgb.Dataset(x_val, label=y_val, reference=lgb_train)

# LightGBM parameters
params = {
    'task': 'train',
    'num_boost_round': 200,
    'early_stopping_rounds': 10,
    'boosting_type': 'gbdt', # (default="gbdt")
    'num_leaves': 300, # (default=31)
    'max_depth': -1, # (default=-1)
    'learning_rate': 0.1, # (default=0.1)
    'n_estimators': 500, # (default=10)
    'max_bin': 30, # (default=255)
    'subsample_for_bin': 100*10000, # (default=50000)
    'objective': 'binary', # (default=None)
    'min_split_gain': 0., # (default=0.)
    'min_child_weight': 1e-3, # (default=1e-3)
    'min_child_samples': 10, # (default=20)
    'subsample': 0.7, # (default=1.)
#     'subsample_freq': 1, # (default=1)
    'colsample_bytree': 0.9, # (default=1.)
    'reg_alpha': 0., # (default=0.)
    'reg_lambda': 0., # (default=0.)
    'random_state': random_num, # (default=None)
    'n_jobs': -1, # (default=-1)
    'silent': False, # (default=True)
    'metric': ['auc', 'binary_logloss'],
}
print('params: ', params)
# train
gbm = lgb.train(
    params,
    train_set=lgb_train,
    valid_sets=lgb_val
)
print('*' * 80)
y_train_proba = gbm.predict(x_train, num_iteration=gbm.best_iteration)
y_train_pred = (y_train_proba>=0.5).astype(int)
acc_train = accuracy_score(y_train, y_train_pred)
roc_train = roc_auc_score(y_train, y_train_proba)
print('acc_train: %.4f \t roc_train: %.4f' % (acc_train, roc_train))

y_val_proba = gbm.predict(x_val, num_iteration=gbm.best_iteration)
y_val_pred = (y_val_proba>=0.5).astype(int)
acc_val = accuracy_score(y_val, y_val_pred)
roc_val = roc_auc_score(y_val, y_val_proba)
print('acc_val:   %.4f \t roc_val:   %.4f' % (acc_val, roc_val))

params:  {'task': 'train', 'num_boost_round': 200, 'early_stopping_rounds': 10, 'boosting_type': 'gbdt', 'num_leaves': 300, 'max_depth': -1, 'learning_rate': 0.1, 'n_estimators': 500, 'max_bin': 30, 'subsample_for_bin': 1000000, 'objective': 'binary', 'min_split_gain': 0.0, 'min_child_weight': 0.001, 'min_child_samples': 10, 'subsample': 0.7, 'colsample_bytree': 0.9, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'random_state': 5537, 'n_jobs': -1, 'silent': False, 'metric': ['auc', 'binary_logloss']}


C:\Users\study\Anaconda3\lib\site-packages\lightgbm\engine.py:99: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\study\Anaconda3\lib\site-packages\lightgbm\engine.py:104: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\study\Anaconda3\lib\site-packages\lightgbm\basic.py:657: UserWarning: silent keyword has been found in `params` and will be ignored. Please use silent argument of the Dataset constructor to pass this parameter.
  'Please use {0} argument of the Dataset constructor to pass this parameter.'.format(key))


[1]	valid_0's auc: 0.870981	valid_0's binary_logloss: 0.598812
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's auc: 0.924831	valid_0's binary_logloss: 0.521414
[3]	valid_0's auc: 0.922467	valid_0's binary_logloss: 0.457006
[4]	valid_0's auc: 0.918642	valid_0's binary_logloss: 0.402577
[5]	valid_0's auc: 0.917044	valid_0's binary_logloss: 0.356097
[6]	valid_0's auc: 0.923019	valid_0's binary_logloss: 0.315931
[7]	valid_0's auc: 0.922941	valid_0's binary_logloss: 0.281112
[8]	valid_0's auc: 0.92184	valid_0's binary_logloss: 0.250861
[9]	valid_0's auc: 0.935541	valid_0's binary_logloss: 0.224258
[10]	valid_0's auc: 0.937987	valid_0's binary_logloss: 0.200922
[11]	valid_0's auc: 0.936971	valid_0's binary_logloss: 0.18037
[12]	valid_0's auc: 0.937797	valid_0's binary_logloss: 0.162075
[13]	valid_0's auc: 0.93698	valid_0's binary_logloss: 0.14595
[14]	valid_0's auc: 0.936865	valid_0's binary_logloss: 0.13162
[15]	valid_0's auc: 0.938767	valid_0's binary_logloss: 0

## Predict

In [11]:
run_name_acc = run_name + '_' + str(int(roc_val*10000)).zfill(4)
print(run_name_acc)

TalkingdataAFD2018_XGBoost_20180423_011444_9612


In [12]:
y_test_proba = gbm.predict(x_test, num_iteration=gbm.best_iteration)
print(y_test_proba.shape)
print(y_test_proba[:20])

(18790469,)
[ 0.00064331  0.00047519  0.00024903  0.00033476  0.00032784  0.00054662
  0.00042919  0.00049023  0.00049017  0.00028977  0.00138054  0.00038791
  0.00025511  0.00126957  0.0002316   0.00024032  0.00049644  0.00024032
  0.00073376  0.00034471]


In [13]:
def save_proba(y_train_proba, y_train, y_val_proba, y_val, y_test_proba, click_ids, file_name):
    print(click_ids[:5])
    if os.path.exists(file_name):
        os.remove(file_name)
        print('File removed: \t%s' % file_name)
    with h5py.File(file_name) as h:
        h.create_dataset('y_train_proba', data=y_train_proba)
        h.create_dataset('y_train', data=y_train)
        h.create_dataset('y_val_proba', data=y_val_proba)
        h.create_dataset('y_val', data=y_val)
        h.create_dataset('y_test_proba', data=y_test_proba)
        h.create_dataset('click_ids', data=click_ids)
    print('File saved: \t%s' % file_name)

def load_proba(file_name):
    with h5py.File(file_name, 'r') as h:
        y_train_proba = np.array(h['y_train_proba'])
        y_train = np.array(h['y_train'])
        y_val_proba = np.array(h['y_val_proba'])
        y_val = np.array(h['y_val'])
        y_test_proba = np.array(h['y_test_proba'])
        click_ids = np.array(h['click_ids'])
    print('File loaded: \t%s' % file_name)
    print(click_ids[:5])
    
    return y_train_proba, y_train, y_val_proba, y_val, y_test_proba, click_ids


y_proba_file = os.path.join(model_folder, 'proba_%s.p' % run_name_acc)
save_proba(y_train_proba, y_train, y_val_proba, y_val, y_test_proba, np.array(sample_submission_csv['click_id']), y_proba_file)
y_train_proba, y_train, y_val_proba, y_val, y_test_proba, click_ids = load_proba(y_proba_file)

print(y_train_proba.shape)
print(y_train.shape)
print(y_val_proba.shape)
print(y_val.shape)
print(y_test_proba.shape)
print(len(click_ids))

[0 1 2 3 4]
File saved: 	D:\bitbucket\kaggle\talkingdata-adtracking-fraud-detection\model\proba_TalkingdataAFD2018_XGBoost_20180423_011444_9612.p
File loaded: 	D:\bitbucket\kaggle\talkingdata-adtracking-fraud-detection\model\proba_TalkingdataAFD2018_XGBoost_20180423_011444_9612.p
[0 1 2 3 4]
(9215482,)
(9215482,)
(93086,)
(93086,)
(18790469,)
18790469


In [14]:
%%time
submission_csv_file = os.path.join(output_folder, 'pred_%s.csv' % run_name_acc)
print(submission_csv_file)
submission_csv = pd.DataFrame({ 'click_id': click_ids , 'is_attributed': y_test_proba })
submission_csv.to_csv(submission_csv_file, index = False)

D:\bitbucket\kaggle\talkingdata-adtracking-fraud-detection\output\pred_TalkingdataAFD2018_XGBoost_20180423_011444_9612.csv
Wall time: 1min 13s


In [15]:
print('Time cost: %.2f s' % (time.time() - t0))

print('random_num: ', random_num)
print('date: ', date)
print(run_name_acc)
print('Done!')

Time cost: 338.22 s
random_num:  5537
date:  6
TalkingdataAFD2018_XGBoost_20180423_011444_9612
Done!
